In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as mb

In [2]:
opsdb = db.OpsimDatabase('minion_1016_newsky.db')

In [52]:
simdata = opsdb.query_columns(opsdb.defaultTable, 
                              colnames=['fieldRA', 'fieldDec', 'filter', 'night', 'visitTime', 'airmass', 'filtSkyBrightness', 'expMJD'], 
                              sqlconstraint=None)

In [53]:
ra_array      = np.zeros(len(simdata))
dec_array     = np.zeros(len(simdata))
airmass_array = np.zeros(len(simdata))
time_array    = np.zeros(len(simdata))
night_array   = np.zeros(len(simdata))

# creating empty array for strings
val           = [''] * len(simdata)
filter_array  = np.array(val)

for i in range(len(simdata)):
    ra_array[i]      = simdata[i][0]
    dec_array[i]     = simdata[i][1]
    filter_array[i]  = simdata[i][2]
    night_array[i]   = simdata[i][3]
    time_array[i]    = simdata[i][7]#[4]
    airmass_array[i] = simdata[i][5]

In [5]:
import sncosmo
from   astropy.table    import Table
import numpy            as np
import matplotlib.pylab as plt
from   astropy.io       import ascii

In [6]:
filename = "/global/homes/a/anitab/DC2_run1p1_TransientDocs/Notebooks/Hosted_SN_table.csv"
from pandas import read_csv

In [7]:
vvv  = read_csv(filename)
data = vvv['galaxy_id c mB t0 x0 x1 z random_Hosting rand_host zbin "Unnamed: 0" diskMassStellar morphology/diskHalfLightRadiusArcsec morphology/diskMinorAxisArcsec morphology/positionAngle morphology/spheroidHalfLightRadiusArcsec morphology/spheroidMinorAxisArcsec size_bulge_true size_disk_true totalMassStellar uniqueId raJ2000_gal decJ2000_gal z zbin_gals snid']

In [ ]:
###vvv.columns

In [8]:
#### SN properties from sampled galaxies ####

length  = len(data)
zvals   = np.zeros(length) 
x0vals  = np.zeros(length)
x1vals  = np.zeros(length)
cvals   = np.zeros(length)
mBvals  = np.zeros(length)
t0vals  = np.zeros(length)
smass   = np.zeros(length)
ravals  = np.zeros(length)
decvals = np.zeros(length)

for i in range(length):
    zvals[i]   = data[i].split()[6]
    x0vals[i]  = data[i].split()[4]
    x1vals[i]  = data[i].split()[5]
    cvals[i]   = data[i].split()[1]
    mBvals[i]  = data[i].split()[2]
    t0vals[i]  = data[i].split()[3]
    smass[i]   = data[i].split()[20]
    ravals[i]  = data[i].split()[21]
    decvals[i] = data[i].split()[22]

### Choosing rows of OpSim based on sampled SNe

In [9]:
import os
import opsimsummary as oss
import healpy as hp
print(oss.__version__)

Some imports failed, which implies some dependencies are missing as described below
No module named 'mpl_toolkits.basemap'
Visulization functions based on maps will not work
1.16.1


In [10]:
from opsimsummary import SynOpSim

In [11]:
myopsimv3 = 'minion_1016_newsky.db'

In [12]:
synopsim = SynOpSim.fromOpSimDB(myopsimv3, opsimversion='lsstv3',
                                angleUnit='degrees', usePointingTree=True)

 reading from database sqlite:///minion_1016_newsky.db
SELECT * FROM Summary WHERE propID in (56, 54)


In [13]:
gen = synopsim.pointingsEnclosing(ravals, decvals, circRadius=0., pointingRadius=1.75, usePointingTree=True)

In [ ]:
#sum(1 for g in gen)
#len(ravals)
#len(rvals) should be equal to the suma above

In [ ]:
#df= next(gen)
#raopsim = df['fieldRA']

In [64]:
#raopsim_list  = []
#decopsim_list = []

timeopsim_list   = []
filteropsim_list = []

for i, df in enumerate(gen):
    #raopsim = df['fieldRA']
    #raopsim_list.append(raopsim.values)
    ###raopsim_list += raopsim.values.tolist()
    
    timeopsim = df['expMJD']
    timeopsim_list.append(timeopsim.values)
    
    filteropsim = df['filter']
    filteropsim_list.append(filteropsim.values)
    
    #decopsim = df['fieldDec']
    #decopsim_list.append(decopsim.values)

In [20]:
#np.shape(raopsim_list)

(27017,)

In [140]:
'''
ifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)
ify      = np.array([1,6,11,16,21])

plt.figure(1, figsize=(15,15))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.plot(np.rad2deg(decopsim_list[1000*i]), np.rad2deg(raopsim_list[1000*i]), ".", color="blue")
    plt.plot(decvals[1000*i], ravals[1000*i], ".", color="red")
    plt.ylim(50,60)
    plt.xlim(-34,-24)
    plt.tick_params(axis='both', labelleft='off', labelbottom='off')
    #plt.title("SN number {0}".format(1000*i))
    if i+1 in ifx:
        plt.xlabel("Ra [deg]",fontsize=15)
        plt.tick_params(axis='x', labelbottom='on')
    if i+1 in ify:
        plt.ylabel("Dec [deg]",fontsize=15)
        plt.tick_params(axis='y', labelleft='on')
    
plt.suptitle("SNe in red with opsim visits in blue")
'''

'\nifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)\nify      = np.array([1,6,11,16,21])\n\nplt.figure(1, figsize=(15,15))\n\nfor i in range(25):\n    plt.subplot(5,5,i+1)\n    plt.plot(np.rad2deg(decopsim_list[1000*i]), np.rad2deg(raopsim_list[1000*i]), ".", color="blue")\n    plt.plot(decvals[1000*i], ravals[1000*i], ".", color="red")\n    plt.ylim(50,60)\n    plt.xlim(-34,-24)\n    plt.tick_params(axis=\'both\', labelleft=\'off\', labelbottom=\'off\')\n    #plt.title("SN number {0}".format(1000*i))\n    if i+1 in ifx:\n        plt.xlabel("Ra [deg]",fontsize=15)\n        plt.tick_params(axis=\'x\', labelbottom=\'on\')\n    if i+1 in ify:\n        plt.ylabel("Dec [deg]",fontsize=15)\n        plt.tick_params(axis=\'y\', labelleft=\'on\')\n    \nplt.suptitle("SNe in red with opsim visits in blue")\n'

In [141]:
'''plt.plot(decopsim_list, raopsim_list, ".", color="blue")'''

'plt.plot(decopsim_list, raopsim_list, ".", color="blue")'

In [42]:
#from lsst.sims.photUtils import Sed, Bandpass, LSSTdefaults, calcMagError_sed

In [60]:
from lsst.sims.catUtils.supernovae import SNObject
def get_flux(i):
    params = {'x0': x0vals[i], 'x1': x1vals[i], 'c': cvals[i]}
    snobject = SNObject(ravals[i], decvals[i])
    snobject.set(**params)
    return snobject.catsimBandFlux(bandpassobject = filter_array[i], time=time_array[i])# time=t0vals[i])

In [303]:
'''
m        = 200
params   = {'x0': x0vals[m], 'x1': x1vals[m], 'c': cvals[m], 't0':t0vals[m]}
snobject = SNObject(ravals[m], decvals[m])
snobject.set(**params)

valsff = np.zeros(len(filteropsim_list[m]))
for j in range(len(filteropsim_list[m])):
    valsff[j] = snobject.catsimBandFlux(bandpassobject = filteropsim_list[m][j], time=timeopsim_list[m][j])
'''
#mlist = []
#jlist = []
jlist_tot = []

for m in range(len(timeopsim_list)):
    jlist = []
    for j in range(len(timeopsim_list[m])):
        if (timeopsim_list[m][j] > t0vals[m] -19 and timeopsim_list[m][j] < t0vals[m] + 49):
            #mlist.append(m)
            jlist.append(j)
    jlist_tot.append(jlist)

In [304]:
np.shape(jlist_tot)

(27017,)

In [309]:
jlist_tot[4]

[]

In [317]:
cc = 0
for xx in range(len(jlist_tot)):
    if not jlist_tot[xx]:
        cc = cc+1

In [318]:
cc

4086

In [269]:
np.shape(t0vals)

(27017,)

In [263]:
valsff.max()

0.0

In [290]:
m        = 500
params   = {'x0': x0vals[m], 'x1': x1vals[m], 'c': cvals[m], 't0':t0vals[m]}
snobject = SNObject(ravals[m], decvals[m])
snobject.set(**params)

In [294]:
snobject.catsimBandFlux(bandpassobject = LSST_BandPass['y'], time = t0vals[m]+49) #time=t0vals[200])

6071838.1841441281

In [297]:
snobject.catsimBandFlux(bandpassobject = LSST_BandPass['y'], time = timeopsim_list[m][200]) #time=t0vals[200])

0.0

In [298]:
timeopsim_list[m][200]

59630.020547

In [300]:
t0vals[m] - timeopsim_list[m][200]

2925.7161072572708

In [264]:
t0vals[200]

62246.982731920325

In [233]:
np.where(np.abs(timeopsim_list[m] - t0vals[m]) < 30)

(array([163, 186]),)

In [237]:
timeopsim_list[m][186] - t0vals[200]

-29.992204920323275

In [173]:
valsff[657]

0.0

In [190]:
t0vals[100]

60542.791631872075

In [166]:
t0vals.min()

59580.157238456311

In [165]:
np.min(timeopsim_list[0])

59583.080872999999

In [187]:
print(snobject)

source:
  class      : SALT2Source
  name       : 'salt2-extended'
  version    : 1.0
  phases     : [-20, .., 50] days
  wavelengths: [300, .., 18000] Angstroms
effect (name='host' frame='rest'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
effect (name='mw' frame='obs'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
parameters:
  z       = 0.0
  t0      = 61171.381392818315
  x0      = 783844055879.03784
  x1      = 1.1963657619373145
  c       = -0.019523983646587639
  hostebv = 0.0
  hostr_v = 3.1000000000000001
  mwebv   = 0.0
  mwr_v   = 3.1000000000000001


In [61]:
flux_vals = np.zeros(len(t0vals))

for i in range(len(t0vals)):
    flux_vals[i] = get_flux(i)

In [ ]:
#t0s = t0vals
#for i, ops in enumerate(gen):
#    t0 = t0s[i]
#    x = ops.query('expMJD > @t0 - 60. and expMJD < @t0 + 90')
#    print ("done")

### LSST code example that works!

In [147]:
bandpassnames = ['u', 'g', 'r', 'i', 'z', 'y']
from lsst.sims.photUtils.BandpassDict import BandpassDict
LSST_BandPass = BandpassDict.loadTotalBandpassesFromFiles()
SN = SNObject(ra=30., dec=-60.)
SN.set(z=0.96, t0=571181, x1=2.66, c=0.353, x0=1.796112e-06)
SN.catsimBandFlux(bandpassobject=LSST_BandPass['r'], time=571190.)

2.0719475911259226e-11

In [148]:
print(snobject)

source:
  class      : SALT2Source
  name       : 'salt2-extended'
  version    : 1.0
  phases     : [-20, .., 50] days
  wavelengths: [300, .., 18000] Angstroms
effect (name='host' frame='rest'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
effect (name='mw' frame='obs'):
  class           : OD94Dust
  wavelength range: [909.09, 33333.3] Angstroms
parameters:
  z       = 0.0
  t0      = 0.0
  x0      = 1.796112e-06
  x1      = 1.1963657619373145
  c       = -0.019523983646587639
  hostebv = 0.0
  hostr_v = 3.1000000000000001
  mwebv   = 0.0
  mwr_v   = 3.1000000000000001


In [92]:
timeopsim_list[0]

array([ 61815.105464,  62466.214044,  61381.080246, ...,  61792.139922,
        61792.157959,  60012.015765])

In [98]:
np.max(x0vals)/1e12

1.7949712914658154

In [110]:
def mB(x0):
    return 10.635 - (2.5 * np.log10(x0))

In [128]:
mBvals = mB(x0vals/1e15)

In [129]:
np.min(mBvals)

17.499856232697002

In [130]:
np.max(mBvals)

19.936523486604049

In [115]:
mB(1.796112e-06)

24.99916646390259

In [146]:
filteropsim_list

[array(['i', 'r', 'u', ..., 'r', 'r', 'z'], dtype=object),
 array(['g', 'r', 'g', ..., 'y', 'i', 'i'], dtype=object),
 array(['u', 'i', 'i', ..., 'y', 'r', 'i'], dtype=object),
 array(['z', 'i', 'i', ..., 'y', 'u', 'y'], dtype=object),
 array(['y', 'y', 'i', ..., 'y', 'r', 'z'], dtype=object),
 array(['i', 'i', 'z', ..., 'g', 'r', 'u'], dtype=object),
 array(['i', 'i', 'i', ..., 'i', 'g', 'i'], dtype=object),
 array(['i', 'y', 'y', ..., 'i', 'z', 'z'], dtype=object),
 array(['i', 'i', 'i', ..., 'i', 'z', 'z'], dtype=object),
 array(['i', 'z', 'i', ..., 'z', 'u', 'z'], dtype=object),
 array(['z', 'y', 'i', ..., 'r', 'i', 'g'], dtype=object),
 array(['i', 'z', 'z', ..., 'r', 'r', 'z'], dtype=object),
 array(['y', 'z', 'i', ..., 'y', 'z', 'y'], dtype=object),
 array(['z', 'z', 'u', ..., 'g', 'z', 'y'], dtype=object),
 array(['i', 'r', 'r', ..., 'y', 'i', 'g'], dtype=object),
 array(['z', 'y', 'i', ..., 'y', 'i', 'g'], dtype=object),
 array(['r', 'z', 'i', ..., 'z', 'z', 'y'], dtype=object